In [13]:
import requests
import json

In [40]:
def parse_nested_json(data):
    """
    Recursively parse nested JSON strings into Python dictionaries.
    """
    if isinstance(data, dict):
        return {key: parse_nested_json(value) for key, value in data.items()}
    elif isinstance(data, list):
        return [parse_nested_json(item) for item in data]
    elif isinstance(data, str):
        try:
            return parse_nested_json(json.loads(data))
        except (json.JSONDecodeError, TypeError):
            return data
    else:
        return data

In [28]:
def get_token(username: str, password: str) -> str:
    """
    Get token from iacpass
    :param username: username
    :param password: password
    :return: token
    """
    headers = {
        "accept": "application/json",
        "Content-Type": "application/json"
    }

    payload = {"username": username, "password": password}
    r = requests.post("https://iacpaas.dvo.ru/api/signin", json=payload, headers=headers)
    if r.status_code != 200:
        raise Exception("Failed to get token: {}".format(r.text))
    return r.json()["accessToken"]


In [37]:
def download_from_repo(name:str, path: str, token: str, json_type: str = "universal", compress: bool = False, no_blob_data: bool = True) -> None:
    """
    Downloads a file from iacpass repository.
    :param name: name of the file to be saved
    :param url: base url to the repository
    :param path: Path to the file
    :param compress: ...
    :param no_blob_data: ...
    :return: None
    """

    params = {"path": path, "json-type": json_type, "compress": compress, "no-blob-data": no_blob_data}

    headers = {"Authorization": f"Bearer {token}"}

    r = requests.get("https://iacpaas.dvo.ru/api/data/export/user-item", params=params, headers=headers)

    if r.status_code == 200:

        response_json = r.json()

        parsed_data = parse_nested_json(response_json)
    
        with open(f"{name}.json", "w", encoding="utf-8") as json_file:
            json.dump(parsed_data, json_file, indent=2, ensure_ascii=False)
    
        print(f"File {name}.json has been saved")
    else:
        raise Exception("Failed to download file")


    

In [32]:
example_path = " Загрузки / Архив протоколов технологических операций лазерной обработки"


In [29]:
token = get_token("some_email@ru", "some_password")

In [39]:
download_from_repo("response", example_path, token)

Response saved to 'response.json'
